# Using DwC-A_dotnet.Interactive

This notebook describes how to use DwC-A_dotnet and DwC-A_dotnet.Interactive to interactively query Darwin Core Archive files and plot your results.

Information on the dotnet libraries used here may be found at 

|Library|Link|
|---|---|
|DwC-A_dotnet|https://github.com/pjoiner/DwC-A_dotnet|
|DwC-A_dotnet.Interactive|https://github.com/pjoiner/DwC-A_dotnet.Interactive|

## Installation

Use the #r magic command to install the libraries from NuGet.

In [1]:
#r "nuget:DwC-A_dotnet,0.5.1"
#r "nuget:DwC-A_dotnet.Interactive,0.1.1-Pre"

## Open An Archive
Use the `ArchiveReader` class to open the archive and provide the path to your archive.  It is recommended that the archive be unzipped to a directory first to reduce the overhead of creating a temporary folder to unzip the archive.  If you use the zip file remember to dispose of the temporary working directory at the end of you session by calling `archive.Dispose();`

In [1]:
using DwC_A;

var archive = new ArchiveReader(@"./data/dwca-rooftop-v1.4");

## Archive MetaData
The interactive extensions library (`DwC-A_dotnet.Interactive`) registers kernel extensions to display various archive metadata by simply entering MetaData at the end of a cell.  The same can be done for an `IFileReader` instance to get a list of the term metadata.

In [1]:
archive.MetaData

In [1]:
archive.CoreFile

In [1]:
archive.Extensions.GetFileReaderByFileName("occurrence.txt")

## ToDynamic Extension Methods
A row may be converted to a `dynamic` object using the `ToDynamic()` function from the `DwC_A.Interactive.Extensions` namespace.  Each field will be accessable by using the short name for the Term for a row.  The following cell displays a subset of event data from the first 5 rows of the dataset.

In [1]:
using DwC_A_dotnet.Interactive.Extensions;

//This will return an IEnumerable<dynamic>
var dynRows = archive.CoreFile.ToDynamic();
foreach(var dynRow in dynRows.Take(5))
{
    var event1 = new 
    {
        dynRow.eventID,
        dynRow.eventDate,
        dynRow.sampleSizeValue
    };
    display(event1);
}

## Query Data Using LinQ

The following cell uses LinQ to gather a list of total individual counts of each genus for a specific sampling event.  Change the number in the `.Skip(1)` line to see totals calculated for other events. 

In [1]:
var eventID = archive.CoreFile.ToDynamic()
    .Skip(1)  //<== Change this number to see other events
    .First()
    .eventID;

display(eventID);

var eventOccurrences = archive.Extensions.GetFileReaderByFileName("occurrence.txt").ToDynamic();    

var data = eventOccurrences
    .Where(n => n.eventID == eventID)
    .GroupBy(n => n.genus)
    .Select(g => new{
        genus = g.First().genus,
        count = g.Sum(c => int.Parse(c.individualCount))
    });
data